In [35]:
import random
import json
import pickle

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD


import numpy as np

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [36]:
intents = json.loads(open("intentsUpdated.json").read())

words = []
classes = []
documents = []

ignore_letters = ["?", "!", ".", ","]

In [37]:
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent["tag"]))

        if intent["tag"] not in classes:
            classes.append(intent["tag"])
words = [lemmatizer.lemmatize(word)
         for word in words if word not in ignore_letters]

In [38]:
words = sorted(set(words))
classes = sorted(set(classes))

In [39]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

dataset = []
template = [0]*len(classes)

In [40]:
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower())
                     for word in word_patterns]

    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(template)
    output_row[classes.index(document[1])] = 1
    dataset.append([bag, output_row])

In [41]:
random.shuffle(dataset)

# Extracting the input (features) and output (target) data
train_x = [data[0] for data in dataset]  # Features
train_y = [data[1] for data in dataset]  # Target labels

In [42]:
# Now, you can convert them into NumPy arrays individually
train_x = np.array(train_x)
train_y = np.array(train_y)

In [49]:
# Scale feature data
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)

In [60]:
# Define the model
model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),),
                activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [61]:
# Compile the model
sgd = SGD(learning_rate=0.01,
          momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=['accuracy'])

hist = model.fit(train_x, train_y,
                 epochs=200, batch_size=5, verbose=1)

Epoch 1/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0100 - loss: 4.8673
Epoch 2/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0323 - loss: 4.8270
Epoch 3/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0268 - loss: 4.7914
Epoch 4/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0280 - loss: 4.7304
Epoch 5/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0314 - loss: 4.6813
Epoch 6/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0466 - loss: 4.5917
Epoch 7/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0928 - loss: 4.4886
Epoch 8/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0891 - loss: 4.3306
Epoch 9/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1241 - loss: 4.1584
Epoch 10/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1778 - loss: 3.9278
Epoch 11/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1771 - loss: 3.7263
Epoch 12/200
144/144 ━━━━━━━━━━━━━━━━━━━━

In [62]:
model.save("chatbot_model.h5", hist)
print("Done!")

Done!
